In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import shutil

C:\Users\CUB\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
pip install tensorflow

     -------------------------------------- 385.0/385.0 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 413.4/413.4 kB 6.4 MB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 kB 8.2 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 6.7 MB/s eta 0:00:00
     -------------------------------------- 126.7/126.7 kB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 26.4/26.4 MB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 15.8/15.8 MB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 3.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [7]:
original_dataset_dir = r"C:\Users\CUB\Downloads\Machine-Learning-Deep-Learning-in-Python-R\practice files\excel or csv\Dataset"

In [6]:
base_dir = r'C:\Users\CUB\Downloads\Machine-Learning-Deep-Learning-in-Python-R\practice files\excel or csv\temp_data'

In [8]:
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')

In [9]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

In [10]:
gesture_classes = os.listdir(original_dataset_dir)
for gesture in gesture_classes:
    os.makedirs(os.path.join(train_dir, gesture), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, gesture), exist_ok=True)

In [12]:
for gesture in gesture_classes:
    gesture_dir = os.path.join(original_dataset_dir, gesture)
    images = os.listdir(gesture_dir)
    train_images, valid_images = train_test_split(images, test_size=0.2, random_state=42)
    for image in train_images:
        shutil.copy(os.path.join(gesture_dir, image), os.path.join(train_dir, gesture))

    for image in valid_images:
        shutil.copy(os.path.join(gesture_dir, image), os.path.join(valid_dir, gesture))

In [13]:
img_width, img_height = 150, 150
batch_size = 32

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [15]:
valid_datagen = ImageDataGenerator(rescale=1.0/255)


In [16]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5280 images belonging to 12 classes.


In [17]:
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1320 images belonging to 12 classes.


In [18]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(gesture_classes), activation='softmax')  # Number of classes
])


C:\Users\CUB\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
epochs = 25

In [21]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/25


C:\Users\CUB\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


165/165 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.4713 - loss: 1.5955 - val_accuracy: 0.8247 - val_loss: 0.5398
Epoch 2/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.9368
Epoch 3/25


C:\Users\CUB\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


165/165 ━━━━━━━━━━━━━━━━━━━━ 120s 712ms/step - accuracy: 0.7516 - loss: 0.7845 - val_accuracy: 0.8239 - val_loss: 0.4518
Epoch 4/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.1764
Epoch 5/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 121s 716ms/step - accuracy: 0.7842 - loss: 0.6213 - val_accuracy: 0.8552 - val_loss: 0.3685
Epoch 6/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.6990
Epoch 7/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 120s 711ms/step - accuracy: 0.8099 - loss: 0.5550 - val_accuracy: 0.8598 - val_loss: 0.3656
Epoch 8/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.2814
Epoch 9/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 120s 711ms/step - accuracy: 0.8397 - loss: 0.4945 - val_accuracy: 0.8925 - val_loss: 0.3079
Epoch 10/25
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.

In [24]:
model.save('hand_gesture_model.h5')